In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random
import pickle
import cv2
import time
import glob
%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import SGD, Adam, RMSprop 
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, Conv3D, MaxPooling3D, Input, concatenate, AveragePooling3D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

In [ ]:
df_train = pd.read_csv(r"/content/drive/My Drive/Upenn_Data/training_data.csv") #refer .csv for cross-validation implementation
df_val = pd.read_csv(r"/content/drive/My Drive/Upenn_Data/validation_data.csv")
column = "CPM_RadPath_2019_ID"
class_name = "class"
input_path = r'/content/drive/My Drive/UPenn_Data/miccai2019-data/CPM-RadPath_2019_Training_Data/Radiology'
input_path_pat = r'/content/drive/My Drive/Pathology/Output'
flair_format = "_flair.nii.gz"
t1_format = "_t1.nii.gz"
t1ce_format = "_t1ce.nii.gz"
t2_format = "_t2.nii.gz"

In [ ]:
train_flair = []
val_flair = []
train_t1 = []
val_t1 = []
train_t1ce = []
val_t1ce = []
train_t2 = []
val_t2 = []
train_pat = []
val_pat = []
class_train = []
class_val = []

In [ ]:
for i in range(0,208):
    data_path = df_train[column][i]
    
    flair_path = os.path.join(input_path,data_path,data_path+flair_format)
    flair_image = nib.load(flair_path)
    flair = flair_image.get_data()
    flair[flair!=0] = ((((flair[flair!=0] - np.mean(flair[flair!=0]))/ np.std(flair[flair!=0]))+ 10)* 100).astype(dtype=np.int16)
    train_flair.append(flair)
    
    t1_path = os.path.join(input_path,data_path,data_path+t1_format)
    t1_image = nib.load(t1_path)
    t1 = t1_image.get_data()
    t1[t1!=0] = ((((t1[t1!=0] - np.mean(t1[t1!=0]))/ np.std(t1[t1!=0]))+ 10)* 100).astype(dtype=np.int16)
    train_t1.append(t1)
    
    t1ce_path = os.path.join(input_path,data_path,data_path+t1ce_format)
    t1ce_image = nib.load(t1ce_path)
    t1ce = t1ce_image.get_data()
    t1ce[t1ce!=0] = ((((t1ce[t1ce!=0] - np.mean(t1ce[t1ce!=0]))/ np.std(t1ce[t1ce!=0]))+ 10)* 100).astype(dtype=np.int16)
    train_t1ce.append(t1ce)
    
    t2_path = os.path.join(input_path,data_path,data_path+t2_format)
    t2_image = nib.load(t2_path)
    t2 = t2_image.get_data()
    t2[t2!=0] = ((((t2[t2!=0] - np.mean(t2[t2!=0]))/ np.std(t2[t2!=0]))+ 10)* 100).astype(dtype=np.int16)
    train_t2.append(t2)
    
    train_class.append(df_train[class_name][i])

In [ ]:
for i in range(0,52):
    data_path = df_val[column][i]
    
    flair_path = os.path.join(input_path,data_path,data_path+flair_format)
    flair_image = nib.load(flair_path)
    flair = flair_image.get_data()
    flair[flair!=0] = ((((flair[flair!=0] - np.mean(flair[flair!=0]))/ np.std(flair[flair!=0]))+ 10)* 100).astype(dtype=np.int16)
    val_flair.append(flair)
    
    t1_path = os.path.join(input_path,data_path,data_path+t1_format)
    t1_image = nib.load(t1_path)
    t1 = t1_image.get_data()
    t1[t1!=0] = ((((t1[t1!=0] - np.mean(t1[t1!=0]))/ np.std(t1[t1!=0]))+ 10)* 100).astype(dtype=np.int16)
    val_t1.append(t1)
    
    t1ce_path = os.path.join(input_path,data_path,data_path+t1ce_format)
    t1ce_image = nib.load(t1ce_path)
    t1ce = t1ce_image.get_data()
    t1ce[t1ce!=0] = ((((t1ce[t1ce!=0] - np.mean(t1ce[t1ce!=0]))/ np.std(t1ce[t1ce!=0]))+ 10)* 100).astype(dtype=np.int16)
    val_t1ce.append(t1ce)
    
    t2_path = os.path.join(input_path,data_path,data_path+t2_format)
    t2_image = nib.load(t2_path)
    t2 = t2_image.get_data()
    t2[t2!=0] = ((((t2[t2!=0] - np.mean(t2[t2!=0]))/ np.std(t2[t2!=0]))+ 10)* 100).astype(dtype=np.int16)
    val_t2.append(t2)
    
    val_class.append(df_val[class_name][i])

In [ ]:
for i in range(0,208):
    df_path = df_train[column][i]
    
    patch_list = open(os.path.join(input_path_pat,df_path,"patch_list.pkl"), "rb")
    sorted_fsd = pickle.load(patch_list)
    patch_list.close()
    temp=[]
    count = 0
    for i in sorted_fsd:
        image = cv2.imread(os.path.join(input_path_pat,df_path,df_path+"_"+str(sorted_fsd[i][0][0])+"_"+str(sorted_fsd[i][0][1])+".png"), cv2.IMREAD_GRAYSCALE)
        new_image = cv2.resize(image, (224,224))
        temp.append(new_image)
        count=+1
        if count >= 8:
            break
    
    temp = np.array(temp).reshape(8,224,224)
    val_pat.append(temp)

In [ ]:
for i in range(0,52):
    df_path = df_val[column][i]
    
    patch_list = open(os.path.join(input_path_pat,df_path,"patch_list.pkl"), "rb")
    sorted_fsd = pickle.load(patch_list)
    patch_list.close()
    temp=[]
    count = 0
    for i in sorted_fsd:
        image = cv2.imread(os.path.join(input_path_pat,df_path,df_path+"_"+str(sorted_fsd[i][0][0])+"_"+str(sorted_fsd[i][0][1])+".png"), cv2.IMREAD_GRAYSCALE)
        new_image = cv2.resize(image, (224,224))
        temp.append(new_image)
        count=+1
        if count >= 8:
            break
    
    temp = np.array(temp).reshape(8,224,224)
    train_pat.append(temp)

In [ ]:
train_class = []
val_class = []

for i in class_train:
    if i == 'A':
        train_class.append(0)
    elif i == 'O':
        train_class.append(1)
    else:
        train_class.append(2)

for i in class_val:
    if i == 'A':
        val_class.append(0)
    elif i == 'O':
        val_class.append(1)
    else:
        val_class.append(2)

train_labels = to_categorical(train_class)
val_labels = to_categorical(val_class)

In [ ]:
train_flair = np.array(train_flair)
val_flair = np.array(val_flair)
train_t1 = np.array(train_t1)
val_t1 = np.array(val_t1)
train_t1ce = np.array(train_t1ce)
val_t1ce = np.array(val_t1ce)
train_t2 = np.array(train_t2)
val_t2 = np.array(val_t2)

train_flair = np.array(train_flair).reshape(208, 80, 96, 64, 1)
val_flair = np.array(val_flair).reshape(52, 80, 96, 64, 1)
train_t1 = np.array(train_t1).reshape(208, 80, 96, 64, 1)
val_t1 = np.array(val_t1).reshape(52, 80, 96, 64, 1)
train_t1ce = np.array(train_t1ce).reshape(208, 80, 96, 64, 1)
val_t1ce = np.array(val_t1ce).reshape(52, 80, 96, 64, 1)
train_t2 = np.array(train_t2).reshape(208, 80, 96, 64, 1)
val_t2 = np.array(val_t2).reshape(52, 80, 96, 64, 1)
train_pat = np.array(train_pat).reshape(208,8,224,224,1)
val_pat = np.array(val_pat).reshape(52,8,224,224,1)

In [ ]:
input_shape1 = Input(shape = (80, 96, 64, 1))
input_shape2 = Input(shape = (80, 96, 64, 1))
input_shape3 = Input(shape = (80, 96, 64, 1))
input_shape4 = Input(shape = (80, 96, 64, 1))
input_shape5 = Input(shape = (8,224,224,1))

In [ ]:
conv1 = Conv3D(32, 3, activation = 'relu')(input_shape1)
pool1 = MaxPooling3D(pool_size = (3, 3, 3))(conv1)
flat1 = Flatten()(pool1)

conv2 = Conv3D(32, 3, activation = 'relu')(input_shape2)
pool2 = MaxPooling3D(pool_size = (3, 3, 3))(conv2)
flat2 = Flatten()(pool2)

conv3 = Conv3D(32, 3, activation = 'relu')(input_shape3)
pool3 = MaxPooling3D(pool_size = (3, 3, 3))(conv3)
flat3 = Flatten()(pool3)

conv4 = Conv3D(32, 3, activation = 'relu')(input_shape4)
pool4 = MaxPooling3D(pool_size = (3, 3, 3))(conv4)
flat4 = Flatten()(pool4)

pConv1 = Conv3D(64, 7, activation = 'relu')(input_shape5)
pConv2 = Conv3D(64, 3, activation = 'relu')(pConv1)
pConv3 = Conv3D(64, 3, activation = 'relu')(pConv2)
pConv4 = Conv3D(64, 3, activation = 'relu')(pConv3)
pConv5 = Conv3D(64, 3, activation = 'relu')(pConv4)
pConv6 = Conv3D(128, 3, activation = 'relu')(pConv5)
pConv7 = Conv3D(128, 3, activation = 'relu')(pConv6)
pConv8 = Conv3D(128, 3, activation = 'relu')(pConv7)
pConv9 = Conv3D(128, 3, activation = 'relu')(pConv8)
pConv10 = Conv3D(256, 3, activation = 'relu')(pConv9)
pConv11 = Conv3D(256, 3, activation = 'relu')(pConv10)
pConv12 = Conv3D(256, 3, activation = 'relu')(pConv11)
pConv13 = Conv3D(256, 3, activation = 'relu')(pConv12)
pConv14 = Conv3D(512, 3, activation = 'relu')(pConv13)
pConv15 = Conv3D(512, 3, activation = 'relu')(pConv14)
pConv16 = Conv3D(512, 3, activation = 'relu')(pConv15)
pConv17 = Conv3D(512, 3, activation = 'relu')(pConv16)
pool5 = AveragePooling3D(pool_size = (3, 3, 3))(pConv17)
flat5 = Flatten()(pool5)

merge = concatenate([flat1,flat2,flat3,flat4,flat5])
hidden = Dense(128, activation = 'relu')(merge)
dropout = Dropout(0.5)(hidden)
output = Dense(3, activation = 'softmax')(dropout)

model = Model(inputs = [input_shape1,input_shape2,input_shape3,input_shape4,input_shape5], outputs = output)
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = Adam(lr = 0.0001),
    metrics = ['accuracy']
)

In [ ]:
timee = time.time()

history = model.fit([train_flair,train_t1,train_t1ce,train_t2,train_pat], train_labels, epochs = 20, batch_size = 8)

print(time.time()-timee)

In [ ]:
model.save('/content/drive/My Drive/Upenn_Data/Models/3class_model.h5')

In [ ]:
results = model.evaluate([val_flair,val_t1,val_t1ce,val_t2,val_pat], val_labels, batch_size = 8)

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['loss'])
plt.title('Training')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Accuracy', 'Loss'], loc='upper left')
plt.show()